In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
import re
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold,StratifiedKFold
from sklearn.metrics import accuracy_score
data_path = '/data/kaggleData/2sigma/'

/opt/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn.neighbors import KNeighborsClassifier as KN
import pickle

## Loading Data

In [3]:
train_df = pd.read_json(data_path + 'normalized_train.json')
test_df = pd.read_json(data_path + 'normalized_test.json')
print train_df.shape
print test_df.shape

(49352, 132)
(74659, 131)


## Choosing features for the KNN method, in order to speed up the algorithm and avoid overfitting 

In [4]:
statistical = []

for feature in test_df.columns:
    if re.match('((manager_id)|(house_type))\S+((mean)|(median)|(min)|(max))',feature) !=None:
        statistical.append(feature)
        
weightFeatures=[i for i in test_df.columns if i not in statistical ]

In [5]:
weightFeatures.remove('another_day')
weightFeatures.remove('another_hour')

In [6]:
len(train_df)

49352

## Setting the weight for each feature by applyng logistic regression

In [7]:
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=StratifiedKFold(train_y,5,shuffle=True,random_state = 2333)

#run the logistic algorithm to do 
#numericals from xgb142 + some new hcc encoding + with_feat from xgb142
cv_scores=[]
models =[]
for dev_index, val_index in KF:
    dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
    dev_X, val_X = dev_set[weightFeatures].as_matrix(), val_set[weightFeatures].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    #random forest us
    lr = LogisticRegression()
    lr.fit(dev_X,dev_y)
    preds = lr.predict_proba(val_X)
    models.append(lr)   
    cv_scores.append(log_loss(val_y, preds))
        

    #print(cv_scores)
print np.mean(cv_scores)

0.588708123194


/opt/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [8]:
coff = np.abs(models[0].coef_) 
for i in [1,2,3,4]:
    coff+=np.abs(models[i].coef_) 
coeff = coff.sum(axis=0)

total_weight={}
total_weight_list=[]
for i in range(len(weightFeatures)):
    total_weight[weightFeatures[i]]=coeff[i]
    total_weight_list.append((weightFeatures[i],coeff[i]))

In [9]:
#setting the feature weight 
#trying knn using the weight from the above regression
knn_train_df = train_df.copy()
knn_test_df = test_df.copy()

for feature_value in total_weight_list:
    knn_train_df[feature_value[0]]*=np.sqrt(feature_value[1])
    knn_test_df[feature_value[0]]*=np.sqrt(feature_value[1])

In [10]:
train_df = knn_train_df
test_df = knn_test_df

## Cv style generating output for training the meta classifier and generateing the output as meta feature

In [12]:
#prepare for training
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=StratifiedKFold(train_y,5,shuffle=True,random_state = 2333)

In [13]:
features = list(weightFeatures)

In [14]:
#4 neighbor, cv-style for meta classifier training
store = data_path+"knn4/"

cv_scores=[]
i=0

for dev_index, val_index in KF: 
    result_dict = {}
    
    dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
       #filter the features
    dev_X, val_X = dev_set[features].as_matrix(), val_set[features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    kn = KN(4)
    kn.fit(dev_X,dev_y)
    preds = kn.predict_proba(val_X)

    #save the pickles for futures use
    pickl_file = store+'knn4-5fold-out-'+str(i)+'.pickle'
    fileObject = open(pickl_file,'wb') 
    pickle.dump(preds,fileObject)   
    fileObject.close()

    loss = log_loss(val_y, preds)
    
    cv_scores.append(loss)
    i+=1
    print'loss for the turn '+str(i)+' is '+str(loss)

print 'The mean of the cv_scores is:'
print np.mean(cv_scores)

loss for the turn 1 is 3.73216428793
loss for the turn 2 is 3.51690516441
loss for the turn 3 is 3.65620188683
loss for the turn 4 is 3.73433829876
loss for the turn 5 is 3.57432523277
The mean of the cv_scores is:
3.64278697414


In [15]:
# running for meta feature
train_X, test_X = train_df[features].as_matrix(), test_df[features].as_matrix()

et = KN(4)
et.fit(train_X,train_y)
preds = et.predict_proba(test_X)


out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_json(store+'kn4-bulk-out.json')

In [16]:
#8 neighbor, cv-style for meta classifier training
store = data_path+"knn8/"

cv_scores=[]
i=0

for dev_index, val_index in KF: 
    result_dict = {}
    
    dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
       #filter the features
    dev_X, val_X = dev_set[features].as_matrix(), val_set[features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    kn = KN(8)
    kn.fit(dev_X,dev_y)
    preds = kn.predict_proba(val_X)

    #save the pickles for futures use
    pickl_file = store+'knn8-5fold-out-'+str(i)+'.pickle'
    fileObject = open(pickl_file,'wb') 
    pickle.dump(preds,fileObject)   
    fileObject.close()

    loss = log_loss(val_y, preds)
    
    cv_scores.append(loss)
    i+=1
    print'loss for the turn '+str(i)+' is '+str(loss)

print 'The mean of the cv_scores is:'
print np.mean(cv_scores)

loss for the turn 1 is 1.90333706761
loss for the turn 2 is 1.83688495811
loss for the turn 3 is 1.87196682533
loss for the turn 4 is 1.92746471344
loss for the turn 5 is 1.91779881511
The mean of the cv_scores is:
1.89149047592


In [17]:
# running for meta feature
train_X, test_X = train_df[features].as_matrix(), test_df[features].as_matrix()

et = KN(8)
et.fit(train_X,train_y)
preds = et.predict_proba(test_X)


out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_json(store+'kn8-bulk-out.json')

In [18]:
#16 neighbor, cv-style for meta classifier training
store = data_path+"knn16/"

cv_scores=[]
i=0

for dev_index, val_index in KF: 
    result_dict = {}
    
    dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
       #filter the features
    dev_X, val_X = dev_set[features].as_matrix(), val_set[features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    kn = KN(16)
    kn.fit(dev_X,dev_y)
    preds = kn.predict_proba(val_X)

    #save the pickles for futures use
    pickl_file = store+'knn16-5fold-out-'+str(i)+'.pickle'
    fileObject = open(pickl_file,'wb') 
    pickle.dump(preds,fileObject)   
    fileObject.close()

    loss = log_loss(val_y, preds)
    
    cv_scores.append(loss)
    i+=1
    print'loss for the turn '+str(i)+' is '+str(loss)

print 'The mean of the cv_scores is:'
print np.mean(cv_scores)

loss for the turn 1 is 1.13724884705
loss for the turn 2 is 1.070207089
loss for the turn 3 is 1.07347600908
loss for the turn 4 is 1.17321972508
loss for the turn 5 is 1.12752517133
The mean of the cv_scores is:
1.11633536831


In [19]:
# running for meta feature
train_X, test_X = train_df[features].as_matrix(), test_df[features].as_matrix()

et = KN(16)
et.fit(train_X,train_y)
preds = et.predict_proba(test_X)


out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_json(store+'kn16-bulk-out.json')

In [20]:
#16 neighbor, cv-style for meta classifier training
store = data_path+"knn32/"

cv_scores=[]
i=0

for dev_index, val_index in KF: 
    result_dict = {}
    
    dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
       #filter the features
    dev_X, val_X = dev_set[features].as_matrix(), val_set[features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    kn = KN(32)
    kn.fit(dev_X,dev_y)
    preds = kn.predict_proba(val_X)

    #save the pickles for futures use
    pickl_file = store+'knn32-5fold-out-'+str(i)+'.pickle'
    fileObject = open(pickl_file,'wb') 
    pickle.dump(preds,fileObject)   
    fileObject.close()

    loss = log_loss(val_y, preds)
    
    cv_scores.append(loss)
    i+=1
    print'loss for the turn '+str(i)+' is '+str(loss)

print 'The mean of the cv_scores is:'
print np.mean(cv_scores)

loss for the turn 1 is 0.785093480922
loss for the turn 2 is 0.76734245118
loss for the turn 3 is 0.773748269132
loss for the turn 4 is 0.826338507712
loss for the turn 5 is 0.774227242632
The mean of the cv_scores is:
0.785349990316


In [21]:
# running for meta feature
train_X, test_X = train_df[features].as_matrix(), test_df[features].as_matrix()

et = KN(32)
et.fit(train_X,train_y)
preds = et.predict_proba(test_X)


out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_json(store+'kn32-bulk-out.json')